In [1]:
import matplotlib.pyplot as plt
import numpy as np

from scipy import special, stats

from code.python.mpmath_integration import quad_phi

from mpmath import *

In [215]:
x = mpf('2')
alpha = mpf('4')
beta = mpf('-1')
mu = mpf('1/4')
delta = mpf('1')

gamma = sqrt(alpha ** 2 - beta ** 2)
omega = sqrt(delta ** 2 + (x-mu)** 2)

x-mu

mpf('1.75')

In [216]:
a = x-mu
b = -beta

In [217]:
def normcdf(x):
    return erfc(-x / sqrt(2)) / 2

In [218]:
t = 2

normcdf(a/sqrt(t) + b * sqrt(t))

mpf('0.99599502883505998514848')

In [219]:
C = sqrt(t / pi) * exp(-a**2 / (2*t)) * exp(-b*a - b**2/2*t)

C * quad(lambda u: exp(-t * u**2) * exp(sqrt(2) * (a + b*t) * u), [0, inf])

mpf('0.99599502883505998514932')

In [220]:
C = exp(-a**2/2/t) / sqrt(2 *pi) * sqrt(t)

s1 = b
s2 = -b**2 * a / 2
s3 = b**3 * (a ** 2 - t) / 6
s4 = -b**4 * (a ** 3 - 3 * a * t) / 24
s5 = b**5 * (a ** 4 - 6 * a**2 * t + 3 * t**2) / 120
s6 = -b**6 * (a ** 5 - 10 * a**3 * t + 15 * a * t ** 2) / 720

S = s1 + s2 + s3 + s4 + s5 + s6


normcdf(a/sqrt(t)) + C * S

mpf('0.98870234639859494355789')

In [221]:
s1, s2, s3, s4, s5, s6

(mpf('1.0'),
 mpf('-0.875'),
 mpf('0.17708333333333333333326'),
 mpf('0.21419270833333333333326'),
 mpf('-0.12809244791666666666674'),
 mpf('-0.019757758246527777777767'))

In [222]:
def P(k, t, a):
    s = 0
    for m in range(int(floor(k / 2)) + 1):
        s += (-1) ** m / factorial(m) / factorial(k - 2*m) * a**(k - 2 * m) / 2 ** m * t ** m
        # print(m, s)

    return factorial(k) * s

In [223]:
k = 4

P(k, t, a)

mpf('-15.371093749999999999973')

In [224]:
def hermite_prob(k, z):
    s = 0
    for m in range(int(floor(k / 2)) + 1):
        s += (-1) ** m / factorial(m) / factorial(k - 2*m) * z**(k - 2 * m) / 2 ** m

    return factorial(k) * s

In [225]:
hermite_prob(k, a/sqrt(t)) * t ** (k/2)

mpf('-15.371093750000000000014')

In [226]:
2**(-k/2) * hermite(k, a/sqrt(2*t)) * t ** (k/2)

mpf('-15.37109375')

In [227]:
S = 0
for k in range(1, 25):
    h = P(k-1, t, a)
    h = 2**(-(k-1)/2) * hermite(k-1, a/sqrt(2*t)) * t ** ((k-1)/2)
    r = b ** k * (-1) ** (k + 1) * h / factorial(k)
    # print(k, h, r)
    S += r

normcdf(a/sqrt(t)) + C * S

mpf('0.99599502880491322358636')

In [228]:
a, a**2 - t, a**3 - 3*a*t, (a ** 4 - 6 * a**2 * t + 3 * t**2),  (a ** 5 - 10 * a**3 * t + 15 * a * t ** 2)

(mpf('1.75'),
 mpf('1.0625'),
 mpf('-5.140625'),
 mpf('-15.37109375'),
 mpf('14.2255859375'))

### Series

In [229]:
mp_result = quad_phi(x, alpha, beta, mu, delta, digits=100)
float(mp_result)

0.999851744108556

In [230]:
I1 = quad_phi(x, alpha, 0, mu, delta, digits=100)
I1

mpf('0.99880104890848187341171')

In [231]:
mp.dps = 20

C = delta * exp(delta * gamma) / sqrt(2 * pi)

I1 = C * quad(lambda t: t **(-3/2) * normcdf(a/sqrt(t)) * exp(-delta**2/2/t - gamma**2/2 * t), [0, inf])
I1 

mpf('0.998619011953207663903')

In [232]:
quad_phi(x, sqrt(alpha**2 - beta**2), 0, mu, delta, digits=100)

mpf('0.99861901195320766390293')

In [233]:
mp.dps = 20

S = 0
for k in range(1, 20):
    h = quad(lambda t: hermite(k-1, a/sqrt(2*t)) * t ** (k/2-3/2) * exp(-omega**2/2/t - gamma**2/2 * t), [0, inf])
    r = b ** k * (-1) ** (k + 1) * h / factorial(k) * 2**(-(k-1)/2)
    print(k, r)
    S += r

I1 + C * S / sqrt(2 *pi), S

1 0.00035983198549215086485
2 -0.00031485298730563202826
3 0.00015051188814810975499
4 -0.000036840640923811052502
5 7.6427099002232584571e-7
6 2.0836398758706084988e-6
7 -3.7044350260240304221e-7
8 -8.2295046831386408699e-8
9 3.0910328487366055456e-8
10 2.5883151889937407825e-9
11 -2.0791923755476133028e-9
12 -4.5632673951974670425e-11
13 1.3064922241920795522e-10
14 -2.2518401819315719999e-12
15 -8.0089730111891986408e-12
16 3.6468217039127755999e-13
17 4.8689192848127589302e-13
18 -3.2696201627498957065e-14
19 -2.9557127599166039303e-14


(mpf('0.9998517441085253174009'), mpf('0.00016107691272363353990722'))

In [234]:
mp.dps = 20

S = 0
for k in range(0, 40):
    h = quad(lambda t: hermite(k, a/sqrt(2*t)) * t ** (k/2-1) * exp(-omega**2/2/t - gamma**2/2 * t), [0, inf])
    r = b ** (k+1) * (-1) ** (k) * h / factorial(k+1) * 2**(-k/2)
    print(k, r)
    S += r

I1 + C * S / sqrt(2 *pi), S

0 0.00035983198549215086485
1 -0.00031485298730563202826
2 0.00015051188814810975499
3 -0.000036840640923811052502
4 7.6427099002232584571e-7
5 2.0836398758706084988e-6
6 -3.7044350260240304221e-7
7 -8.2295046831386408699e-8
8 3.0910328487366055456e-8
9 2.5883151889937407825e-9
10 -2.0791923755476133028e-9
11 -4.5632673951974670425e-11
12 1.3064922241920795522e-10
13 -2.2518401819315719999e-12
14 -8.0089730111891986408e-12
15 3.6468217039127755999e-13
16 4.8689192848127589302e-13
17 -3.2696201627498957065e-14
18 -2.9557127599166039303e-14
19 2.4861371019576654689e-15
20 1.7970283933517185721e-15
21 -1.7539141725943017405e-16
22 -1.095582451828862466e-16
23 1.1877281368929492167e-17
24 6.700680199627043386e-18
25 -7.8476705225696752439e-19
26 -4.1115118790660878586e-19
27 5.1046969834173648224e-20
28 2.5307318679059536564e-20
29 -3.2864886992813635299e-21
30 -1.5623443302033831646e-21
31 2.1013852287622937249e-22
32 9.6718425633298406395e-23
33 -1.3374307306990161428e-23
34 -6.002893045

(mpf('0.99985174410855604958412'), mpf('0.00016107691272764920957775'))

In [235]:
float(mp_result)

0.999851744108556

##### 1F1

In [236]:
hermite(k, a/sqrt(2*t))

mpf('-5.5242661377261105181907e+28')

In [239]:
f1 = 1/mp.gamma((1 - k)/2) * hyp1f1(-k/2, 1/2, a^2/4/t)
f2 = 2 * a/sqrt(2*t) /mp.gamma(-k/2) * hyp1f1((1-k)/2, 3/2, a^2/4/t)

2**k * sqrt(pi) * (f1 + f2)

ValueError: gamma function pole

Experimental

In [ ]:
k = 4

P(k, t, a)

In [ ]:
s = 0
for m in range(int(floor(k / 2)) + 1):
    s += (-1) ** m / factorial(m) / factorial(k - 2*m) * (1/2/(a/sqrt(t))**2) ** m
    # print(m, s)

factorial(k) * s * a ** k

In [ ]:
def hermite_prob(k, z):
    s = 0
    for m in range(int(floor(k / 2)) + 1):
        s += (-1) ** m / factorial(m) / factorial(k - 2*m) * (1/2/z**2) ** m
    
    return s

In [ ]:
def hermitep(k, z):
    s = 0
    for m in range(int(floor(k / 2)) + 1):
        s += (-1) ** m / factorial(m) / factorial(k - 2*m) * (1/2/z**2) ** m
    
    return s

In [ ]:
factorial(k - 1)

In [ ]:
factorial(k) * a ** k * hermite_prob(k, a/sqrt(t))

In [ ]:
a, a**2 - t, a**3 - 3*a*t, (a ** 4 - 6 * a**2 * t + 3 * t**2)

In [ ]:
normcdf(a/sqrt(t)) + C * S

In [ ]:
exp(-a**2/2/t) / sqrt(2 *pi) * sqrt(t) * sum((b**2 *a/sqrt(2))**k * (-1)**(k+1) * hermite(k, a/sqrt(2*t)) for k in range(1, 10))